# 03 – Hydrological Impact Simulation

This notebook:
1. Assigns building types and psi (runoff coefficient) values
2. Computes stormwater runoff for the **status quo** (current green-roof coverage)
3. Computes runoff for a **50 % green-roof scenario**
4. Compares the two scenarios and visualises the runoff reduction

In [ ]:
import sys
sys.path.insert(0, '..')

import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

from src.simulation.hydrology import (
    PSI_VALUES,
    compute_runoff,
    compute_status_quo_runoff,
    compute_scenario_runoff,
    compare_scenarios,
)

## 1  Paths & simulation parameters

In [ ]:
CLASSIFIED_BLDG = '../data/results/classified_buildings.gpkg'  # From notebook 02

# Design storm: 30-minute event with 20.4 mm precipitation (Berlin r(0.5, 0.5) event)
RAINFALL_DEPTH_MM = 20.4

# Scenario: raise all roofs to at least 50 % green coverage
TARGET_GREEN_FRACTION = 0.50

OUTPUT_SUMMARY = '../data/results/runoff_comparison.csv'

## 2  Load classified buildings & assign building types

In [ ]:
if os.path.isfile(CLASSIFIED_BLDG):
    gdf = gpd.read_file(CLASSIFIED_BLDG)
    print(f'Loaded {len(gdf)} buildings')
    print(gdf[['roof_type', 'building_area_m2', 'green_roof_fraction']].head())
else:
    print('Classified buildings not found – generating synthetic demo data.')
    import numpy as np
    rng = np.random.default_rng(42)
    building_types = list(PSI_VALUES.keys())
    n = 200
    gdf = pd.DataFrame({
        'building_type': rng.choice(building_types, n),
        'building_area_m2': rng.uniform(100, 2000, n),
        'green_roof_fraction': rng.uniform(0, 0.3, n),
    })

In [ ]:
# Map roof_type → building_type key used in PSI_VALUES
# Adjust this mapping to match your actual OSM building tags
ROOF_TO_BTYPE = {
    'flat': 'flat_residential',
    'pitched': 'pitched_residential',
    'other': 'other',
    'unknown': 'other',
}

if 'building_type' not in gdf.columns:
    gdf['building_type'] = gdf['roof_type'].map(ROOF_TO_BTYPE).fillna('other')

print(gdf['building_type'].value_counts())

## 3  Inspect PSI (runoff coefficient) values

In [ ]:
psi_df = pd.DataFrame(PSI_VALUES).T.reset_index()
psi_df.columns = ['building_type', 'psi_conventional', 'psi_green_roof']
print(psi_df.to_string(index=False))

## 4  Run status-quo & scenario simulations

In [ ]:
# Status quo
df_sq = compute_status_quo_runoff(
    buildings=gdf,
    rainfall_depth_mm=RAINFALL_DEPTH_MM,
    building_type_col='building_type',
    area_col='building_area_m2',
    green_roof_fraction_col='green_roof_fraction',
)

# 50 % scenario
df_sc = compute_scenario_runoff(
    buildings=df_sq,
    rainfall_depth_mm=RAINFALL_DEPTH_MM,
    target_green_roof_fraction=TARGET_GREEN_FRACTION,
    building_type_col='building_type',
    area_col='building_area_m2',
    green_roof_fraction_col='green_roof_fraction',
)

total_sq = df_sc['runoff_m3_status_quo'].sum()
total_sc = df_sc['runoff_m3_scenario'].sum()
reduction = total_sq - total_sc
pct       = reduction / total_sq * 100 if total_sq > 0 else 0

print(f'Status quo runoff:     {total_sq:,.1f} m³')
print(f'50 % scenario runoff:  {total_sc:,.1f} m³')
print(f'Runoff reduction:      {reduction:,.1f} m³  ({pct:.1f} %)')

## 5  Comparison summary per building type

In [ ]:
summary = compare_scenarios(
    buildings=gdf,
    rainfall_depth_mm=RAINFALL_DEPTH_MM,
    target_green_roof_fraction=TARGET_GREEN_FRACTION,
    building_type_col='building_type',
    area_col='building_area_m2',
    green_roof_fraction_col='green_roof_fraction',
    group_by_col='building_type',
)
print(summary.to_string(index=False))

os.makedirs(os.path.dirname(OUTPUT_SUMMARY), exist_ok=True)
summary.to_csv(OUTPUT_SUMMARY, index=False)
print(f'\nSummary saved to: {OUTPUT_SUMMARY}')

## 6  Visualise results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# --- Bar chart: status quo vs scenario per building type ---
x = range(len(summary))
width = 0.35
axes[0].bar([i - width / 2 for i in x], summary['total_runoff_status_quo_m3'],
            width, label='Status Quo', color='steelblue')
axes[0].bar([i + width / 2 for i in x], summary['total_runoff_scenario_m3'],
            width, label='50 % Scenario', color='seagreen')
axes[0].set_xticks(list(x))
axes[0].set_xticklabels(summary['group'], rotation=30, ha='right')
axes[0].set_ylabel('Total Runoff (m³)')
axes[0].set_title(f'Runoff Comparison – {RAINFALL_DEPTH_MM} mm Event')
axes[0].legend()

# --- Bar chart: runoff reduction % per building type ---
axes[1].bar(summary['group'], summary['runoff_reduction_pct'], color='darkorange')
axes[1].set_xticklabels(summary['group'], rotation=30, ha='right')
axes[1].set_ylabel('Runoff Reduction (%)')
axes[1].set_title('Runoff Reduction by Building Type')
axes[1].set_ylim(0, 60)

plt.tight_layout()
plt.show()

## 7  Single-building runoff formula demonstration

In [ ]:
# Demonstration: compute_runoff(area_m2, rainfall_depth_mm, psi)
examples = [
    ('Conventional flat roof (500 m²)', 500, RAINFALL_DEPTH_MM, 0.90),
    ('Extensive green roof (500 m²)',   500, RAINFALL_DEPTH_MM, 0.30),
    ('Intensive green roof (500 m²)',   500, RAINFALL_DEPTH_MM, 0.10),
]

for desc, area, rain, psi in examples:
    vol = compute_runoff(area, rain, psi)
    print(f'{desc}: V = {area} m² × {rain} mm × {psi} = {vol:.2f} m³')